In [9]:
import numpy as np
import random
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

# Определение параметров рынка
num_buyers = 3
num_sellers = 2
num_rounds = 20

# Инициализация начальных цен продавцов и начальных ставок покупателей
initial_prices = [random.randint(50, 150) for _ in range(num_sellers)]
initial_bids = [random.randint(30, 100) for _ in range(num_buyers)]

# Определение лимитов ставок и цен для каждого участника
buyer_limits = [random.randint(100, 200) for _ in range(num_buyers)]
buyer_budgets = [random.randint(200, 500) for _ in range(num_buyers)]
seller_costs = [random.randint(20, 80) for _ in range(num_sellers)]

# Коэффициенты адаптации
adaptation_rate = 0.1

# Уровень внешнего фактора (например, экономическая ситуация)
external_factor_index = 1.0

# Определение стратегий участников: консервативная или агрессивная
buyer_strategies = ["conservative", "aggressive", "adaptive"]
seller_strategies = ["aggressive", "adaptive"]

# Функция для обновления ставки или цены
def update_strategy(current, target, rate):
    return current + rate * (target - current)

# Функция для учета спроса и предложения
def adjust_rate_based_on_market(demand, supply):
    if demand > supply:
        return adaptation_rate * 1.5  # Увеличиваем скорость адаптации, если спрос превышает предложение
    elif demand < supply:
        return adaptation_rate * 0.75  # Уменьшаем скорость адаптации, если предложение превышает спрос
    return adaptation_rate

# Функция для изменения адаптации в зависимости от стратегии
def get_adaptation_rate(strategy, base_rate):
    if strategy == "conservative":
        return base_rate * 0.5  # Консервативные участники медленнее адаптируются
    elif strategy == "aggressive":
        return base_rate * 1.5  # Агрессивные участники быстрее адаптируются
    elif strategy == "adaptive":
        return base_rate  # Адаптивные участники подстраиваются динамически
    return base_rate

# Q-Learning модель
def create_q_model(input_shape, action_space):
    model = Sequential()
    model.add(Dense(24, input_dim=input_shape, activation="relu"))
    model.add(Dense(24, activation="relu"))
    model.add(Dense(action_space, activation="linear"))
    model.compile(loss="mse", optimizer=Adam(learning_rate=0.01))
    return model

# Параметры Q-learning
state_size = 3  # Цена продавца, ставка покупателя, внешний фактор
action_size = 5  # Возможные изменения цены/ставки (-2, -1, 0, 1, 2)
q_model = create_q_model(state_size, action_size)

# Проведение нескольких раундов торгов
buyer_bids = initial_bids[:]
seller_prices = initial_prices[:]

for round_num in range(1, num_rounds + 1):
    print(f"\nРаунд {round_num}:")

    # Подсчет спроса и предложения с учетом внешнего фактора
    total_demand = sum(buyer_bids) * external_factor_index
    total_supply = sum(seller_prices) * external_factor_index
    current_adaptation_rate = adjust_rate_based_on_market(total_demand, total_supply)

    # Покупатели делают ставки
    for i in range(num_buyers):
        if buyer_budgets[i] > 0:
            # Получение адаптации с учетом стратегии покупателя
            individual_rate = get_adaptation_rate(buyer_strategies[i], current_adaptation_rate)
            # Покупатель делает ставку, стремясь остаться ниже своего лимита и учитывая бюджет
            
            upper_limit = max(30, int(min(buyer_limits[i], buyer_budgets[i])))
            target_bid = random.randint(30, upper_limit)

            buyer_bids[i] = update_strategy(buyer_bids[i], target_bid, individual_rate)
            buyer_bids[i] = min(buyer_bids[i], buyer_limits[i], buyer_budgets[i])  # Ставка не может превышать лимит и бюджет

            # Q-learning для адаптации ставок
            state = np.array([seller_prices[i % num_sellers], buyer_bids[i], external_factor_index])
            action = np.argmax(q_model.predict(state.reshape(1, -1), verbose=0))
            buyer_bids[i] += action - 2  # Действие изменяет ставку (-2 до +2)

    # Продавцы устанавливают цены
    for i in range(num_sellers):
        # Получение адаптации с учетом стратегии продавца
        individual_rate = get_adaptation_rate(seller_strategies[i], current_adaptation_rate)
        # Продавец устанавливает цену, ориентируясь на себестоимость и возможную прибыль
        target_price = random.randint(seller_costs[i] + 10, seller_costs[i] * 2)
        seller_prices[i] = update_strategy(seller_prices[i], target_price, individual_rate)
        seller_prices[i] = max(seller_prices[i], seller_costs[i])  # Цена не может быть ниже себестоимости

        # Q-learning для адаптации цен
        state = np.array([seller_prices[i], buyer_bids[i % num_buyers], external_factor_index])
        action = np.argmax(q_model.predict(state.reshape(1, -1), verbose=0))
        seller_prices[i] += action - 2  # Действие изменяет цену (-2 до +2)

    # Сопоставление покупателей и продавцов (переговоры)
    transactions = []
    for buyer_index, bid in enumerate(buyer_bids):
        for seller_index, price in enumerate(seller_prices):
            if bid >= price:
                negotiated_price = (bid + price) / 2  # Переговоры для определения окончательной цены
                if buyer_budgets[buyer_index] >= negotiated_price:
                    transactions.append((buyer_index, seller_index, negotiated_price))
                    buyer_budgets[buyer_index] -= negotiated_price  # Списываем средства покупателя
                    break  # Покупатель нашел продавца и завершил сделку

    # Печать результатов текущего раунда
    for buyer_index, seller_index, price in transactions:
        print(f"Покупатель {buyer_index + 1} купил у Продавца {seller_index + 1} по цене {price:.2f}")

    print("Ставки покупателей:", [round(bid, 2) for bid in buyer_bids])
    print("Цены продавцов:", [round(price, 2) for price in seller_prices])
    print("Бюджеты покупателей:", [round(budget, 2) for budget in buyer_budgets])

    # Адаптация внешнего фактора (например, изменения экономической ситуации)
    external_factor_index = external_factor_index * random.uniform(0.9, 1.1)

# Итоговые стратегии после всех раундов
print("\nИтоговые стратегии:")
for i in range(num_buyers):
    print(f"Покупатель {i + 1}: конечная ставка {round(buyer_bids[i], 2)}, оставшийся бюджет {round(buyer_budgets[i], 2)} (лимит {buyer_limits[i]})")

for i in range(num_sellers):
    print(f"Продавец {i + 1}: конечная цена {round(seller_prices[i], 2)} (себестоимость {seller_costs[i]})")



Раунд 1:
Покупатель 1 купил у Продавца 2 по цене 77.26
Покупатель 2 купил у Продавца 2 по цене 73.34
Ставки покупателей: [84.08, 76.22, 58.05]
Цены продавцов: [114.48, 70.45]
Бюджеты покупателей: [137.74, 328.66, 204]

Раунд 2:
Покупатель 1 купил у Продавца 2 по цене 74.63
Покупатель 2 купил у Продавца 2 по цене 72.74
Ставки покупателей: [82.67, 78.9, 65.19]
Цены продавцов: [104.04, 66.58]
Бюджеты покупателей: [63.11, 255.92, 204]

Раунд 3:
Покупатель 2 купил у Продавца 2 по цене 66.28
Покупатель 3 купил у Продавца 2 по цене 68.33
Ставки покупателей: [61.11, 70.62, 74.71]
Цены продавцов: [100.68, 61.95]
Бюджеты покупателей: [63.11, 189.64, 135.67]

Раунд 4:
Покупатель 2 купил у Продавца 2 по цене 74.01
Покупатель 3 купил у Продавца 2 по цене 63.28
Ставки покупателей: [57.15, 88.06, 66.61]
Цены продавцов: [97.85, 59.95]
Бюджеты покупателей: [63.11, 115.63, 72.39]

Раунд 5:
Покупатель 2 купил у Продавца 2 по цене 66.79
Покупатель 3 купил у Продавца 2 по цене 58.39
Ставки покупателей: [5